In [22]:
import cv2
import time
import numpy as np

In [24]:
def region_of_interest(frame, region_points):
    mask = np.zeros_like(frame) 
    cv2.fillPoly(mask, region_points, 255)
    masked_image = cv2.bitwise_and(frame, mask)
    return masked_image

In [26]:
def draw_the_lines(frame, lines):
    lines_frame = np.zeros((frame.shape[0], frame.shape[1], 3), dtype = np.uint8)

    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(lines_frame, (x1, y1), (x2, y2), (255, 0, 0), thickness = 3)

        frame_with_lines = cv2.addWeighted(frame, 0.8, lines_frame, 1, 0.0)

    return frame_with_lines

In [28]:
def get_detected_lanes(frame):
    height, width = frame.shape[0], frame.shape[1]
    
    # turn the image into grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # if gradient below the thresold1, frame is not edge. 
    # if gradient equal or above the thresold1, frame is edge.
    # if gradient between threshold1 and threshold2, only if it is connected 
    canny_frame = cv2.Canny(gray_frame, 100, 120)


    # we are interested in lower region of the frame (driving lines) 
    region_of_interest_vertices = [
        (0, height),
        (width // 2, height * 0.65),
        (width, height)
        
    ]

    cropped_frame = region_of_interest(canny_frame, np.array([region_of_interest_vertices], np.int32))

    lines = cv2.HoughLinesP(cropped_frame, rho = 2, theta = np.pi/180, threshold = 50, lines = np.array([]), 
                          minLineLength = 40, maxLineGap = 150)

    image_with_lines = draw_the_lines(frame, lines)
    
    return image_with_lines

In [30]:
video = cv2.VideoCapture("lane_detection_video.mp4")

while video.isOpened():

    is_grabbed, frame = video.read()
    if is_grabbed:
        frame = get_detected_lanes(frame)
        cv2.imshow("Lane Detection Video", frame)
        cv2.waitKey(20)
    else:
        break
        
video.release()
cv2.destroyAllWindows()